# Benford for 3D MRI - Training data generation

This scripts generates score list using all repositories by each performance measures.
Therefore you need to select the performance measures option you want to run: 0 = MSE, 1 = MAE, 2 = R2

Before to run this script you must to put the xlsx files with the result of the CrossValidation results in the path '../input/input_to_do_score
The output is a xlsx file with the Metrics and the Score.

In [1]:
%reset

In [5]:
# Import the relevant libraries
import pandas as pd
import os

# SELECT THE MEASURE OPTION: 
sheet_name = 2 # 0 = MSE, 1 = MAE, 2 = R2

#Path
table_switch = {
        '0': 'MSE',
        '1': 'MAE',
        '2': 'R2'
    }

nameOutput = table_switch.get(str(sheet_name))
inputDir = ('../output/input_to_do_score') 
outputDir = '../output/Score_' + nameOutput

In [8]:
import os

# Create/check the directory to output
directory1 = outputDir

if os.path.isdir(directory1):
    print("Directory '% s' was already created" % directory1)
else:
    try:
        os.mkdir(directory1)
    except OSError:
        print("Directory '% s' failed" % directory1)
    else:
        print("Directory '% s' created" % directory1)

Directory '../output/Score_R2' created


In [2]:
def process_data(filename):
    
    # Read dataset
    original_data = pd.read_excel(os.path.join(inputDir, filename), sheet_name=sheet_name)

    # Create dataframes
    column_names = ["Metrics", "Score"]
    df_data = pd.DataFrame(columns = column_names)
    # Write in the 'Metrics' colum the metrics of the excel files ('Unnamed: 7' column)
    df_data['Metrics'] = original_data['Unnamed: 0']
    
    # size
    size = original_data.iloc[:,1].size
    print(f"Size: {size}")
    # Create a list started in the most high value (xx) to score the metrics' appearance
    list_score = list(reversed(range(1, df_data.shape[0]+1)))

    # Assign list to 'Score' column
    new_df_data = df_data.assign(Score = list_score)
    print(new_df_data.iloc[0:3])

    # Sort the dataset by 'Metrics' to have the same order in all dataset
    sorted_df_data = new_df_data.sort_values('Metrics')

    return sorted_df_data

In [ ]:
# Directory path
directory = ('../output/input_to_do_score') 

# List all files in the directory
files = os.listdir(directory)

df_list = []
name_list = []
size = 120

# Loop through the filenames
for filename in files:
    # Do something with each filename
    print(filename)
    dataset_processed = process_data(filename)
    df_list.append(dataset_processed)
    name_list.append(filename)

In [ ]:
# Sum the score values for each metric
newList = []

#Choose the dataset you want to use to do the score. 

# 1.5 Tesla: [0], [1], [2], [5], and [6]
for i in range(size):
	newList.append(df_list[0]['Score'].values.tolist()[i] +
                      df_list[1]['Score'].values.tolist()[i] +
                      df_list[2]['Score'].values.tolist()[i] +
                      df_list[5]['Score'].values.tolist()[i] +
                      df_list[6]['Score'].values.tolist()[i])
	
"""
# # For 3 Tesla: [3],  [4], and  [7]
for i in range(size):
	newList.append(df_list[3]['Score'].values.tolist()[i] +
                      df_list[4]['Score'].values.tolist()[i] +
                      df_list[7]['Score'].values.tolist()[i])
"""

print(newList)

In [132]:
# Create a new dataframe to save the Score list 
column_names = ["Metrics", "Score"]
score_df = pd.DataFrame(columns = column_names)
score_df['Metrics'] = df_list[0]['Metrics'].values.tolist()

score_df.iloc[0:3]
score_df['Score'] = newList
score_df.iloc[0:3]

score_sorted = score_df.sort_values('Score', ascending=False)

# Save the score list
score_sorted.to_excel(os.path.join(outputDir, "Score" + nameOutput + ".xlsx"), index=False)